# Table Football Rating Adjustment System

This document explains the methodology behind a rating update system for table football matches. The system adjusts individual player ratings (scaled from 0 to 10) based on match outcomes and individual performance relative to the team average.

## 1. Team Average Calculation

For each team, compute the **team average rating** using the ratings of the goalkeeper and attacker:

$$
\text{TeamAvg} = \frac{\text{Rating}_{\text{goalkeeper}} + \text{Rating}_{\text{attacker}}}{2}
$$

**Example:**

If Team A has:  
- Goalkeeper rating: 2.0  
- Attacker rating: 7.0

Then:

$$
\text{TeamA\_Avg} = \frac{2.0 + 7.0}{2} = 4.5
$$

---

## 2. Expected Outcome Calculation

Using the team average ratings, calculate the **expected win probability** for each team with a logistic function (similar to the Elo rating system).

For Team A:

$$
E_A = \frac{1}{1 + 10^{\frac{R_B - R_A}{d}}}
$$

Where:
- \( R_A \) and \( R_B \) are the average ratings for Team A and Team B.
- \( d \) is a scaling factor adjusting the sensitivity to rating differences.

For Team B, the expected outcome is:

$$
E_B = 1 - E_A
$$

---

## 3. Actual Match Outcome and Goal Margin

### Actual Outcome

Determine the actual match outcome (\(S\)) from the scores:
- **Win:** \( S = 1 \)
- **Loss:** \( S = 0 \)
- **Tie:** \( S = 0.5 \) for both teams

### Goal Difference

Calculate the **goal difference**:

$$
\text{Goal Difference} = |\text{score}_A - \text{score}_B|
$$

### Margin Multiplier

Compute the **margin multiplier (M)** using the logarithm of the goal difference plus one:

$$
M = \ln(\text{Goal Difference} + 1)
$$

This logarithmic function dampens the impact of large goal differences. For a tie ((\text{Goal Difference} = 0\)), set \( M = 1 \) to avoid a zero multiplier.

---

## 4. Team-Level Rating Change

Calculate a **base rating change** for the entire team:

$$
\Delta R_{\text{team}} = K \times M \times (S - E)
$$

Where:
- \( K \) is a constant that controls the maximum rating change.
- \( M \) is the margin multiplier.
- \( S \) is the actual match outcome.
- \( E \) is the expected win probability.

---

## 5. Individual Rating Adjustment

To reflect differences among team members, adjust the team delta for each player based on the difference between their individual rating and the team average.

### 5.1. Adjustment Factor

Calculate an **adjustment factor** for each player:

$$
\text{Adjustment Factor} = 1 + \beta \times (\text{TeamAvg} - \text{PlayerRating})
$$

- **\( \beta \)** is a tuning parameter controlling the sensitivity of the adjustment.
- If a player's rating is **below** the team average (\(\text{TeamAvg} - \text{PlayerRating} > 0\)), the factor is **greater than 1**. On a winning team, this player gains more points.
- Conversely, if a player's rating is **above** the team average, the factor is less than 1 (or more negative on a losing team), causing a larger point loss.

### 5.2. Role Weight and Rating Update

Each player's new rating is updated as follows:

$$
\text{NewRating} = \text{OldRating} + \Delta R_{\text{team}} \times \text{Adjustment Factor} \times w_{\text{role}}
$$

Where:
- \( w_{\text{role}} \) is a weight factor specific to the player's role (e.g., goalkeeper or attacker).

### 5.3. Clipping the Rating

Ensure the updated rating stays within the 0 to 10 range:

$$
\text{NewRating} = \max(0, \min(10, \text{NewRating}))
$$

---

## 6. Overall Process Summary

1. **Team Average Calculation:**  
   Compute the average rating for each team:
   
   $$
   \text{TeamAvg} = \frac{\text{Rating}_{\text{goalkeeper}} + \text{Rating}_{\text{attacker}}}{2}
   $$

2. **Expected Outcome Calculation:**  
   Use the logistic formula to compute each team's expected win probability:
   
   $$
   E_A = \frac{1}{1 + 10^{\frac{R_B - R_A}{d}}}, \quad E_B = 1 - E_A
   $$

3. **Actual Match Outcome and Margin Multiplier:**  
   - Determine the match outcome \( S \) (1 for win, 0 for loss, 0.5 for tie).  
   - Calculate the goal difference and compute the multiplier:
     
     $$
     M = \ln(\text{Goal Difference} + 1)
     $$

4. **Team-Level Rating Change:**  
   Calculate the team delta:
   
   $$
   \Delta R_{\text{team}} = K \times M \times (S - E)
   $$

5. **Individual Rating Adjustment:**  
   - Compute the adjustment factor for each player:
     
     $$
     \text{Adjustment Factor} = 1 + \beta \times (\text{TeamAvg} - \text{PlayerRating})
     $$
   - Update each player's rating with:
     
     $$
     \text{NewRating} = \text{OldRating} + \Delta R_{\text{team}} \times \text{Adjustment Factor} \times w_{\text{role}}
     $$
   - Clip the rating to ensure it remains between 0 and 10:
     
     $$
     \text{NewRating} = \max(0, \min(10, \text{NewRating}))
     $$

This methodology ensures that within the same team, players with different initial ratings will have their adjustments tailored to better reflect their relative performance compared to the team’s overall strength.


In [33]:
import math
import numpy as np

In [ ]:
def update_ratings(team_A, team_B, score_A, score_B, K=0.35, d=1.0, beta=0.1, role_weights= {'goalkeeper': 1.0, 'attacker': 1.0}):
    """
    Update player ratings for a table football match with individual adjustments.
    Each team consists of 2 players with roles 'goalkeeper' and 'attacker'. Ratings are scaled from 0 to 10.

    Parameters:
    - team_A: list, e.g., [2.0, 7.0]
    - team_B: list, e.g., [8.5, 6.0]
    - score_A: int, goals scored by Team A
    - score_B: int, goals scored by Team B
    - K: float, constant determining maximum change per match (default: 0.5)
    - d: float, scaling factor for the expected outcome formula (default: 1.0)
    - beta: float, adjustment factor for individual rating difference (default: 0.1)
    - role_weights: dict, optional custom weights for each role,ì
    """
    
    # Step 1: Calculate the team average ratings.
    team_A_avg = np.mean(team_A)
    team_B_avg = np.mean(team_B)

    # Step 2: Calculate the expected outcome using a logistic function.
    exponent = (team_B_avg - team_A_avg) / d
    E_A = 1 / (1 + math.pow(10, exponent))
    E_B = 1 - E_A

    # Step 3: Determine the actual match result.
    S_A = 1 if score_A > score_B else 0 if score_B > score_A else 0.5
    S_B = 1 - S_A if S_A != 0.5 else 0.5
    
    goal_difference = abs(score_A - score_B)
    
    # Step 4: Compute the margin multiplier M using the logarithm of the goal difference plus one.
    # Use M = 1 if there is no goal difference (to avoid a multiplier of 0).
    M = math.log(goal_difference + 1) if goal_difference > 0 else 1

    # Step 5: Calculate the base team rating change using the formula:
    # ΔR_team = K * M * (S - E)
    delta_team_A = K * M * (S_A - E_A)
    delta_team_B = K * M * (S_B - E_B)
    
    # Step 6: Update each player's rating individually.
    # We adjust each player's change based on the difference between the team average and the player's rating.
    # For the winning team, players below the average get an extra boost, while those above get less.
    # For the losing team, players above the average lose more, while those below lose less.

    # Calculate adjustment factors
    adjustment_factors_A = 1 + beta * (team_A_avg - team_A)
    adjustment_factors_B = 1 + beta * (team_B_avg - team_B)
    
    # Inverse the adjustment factor for the losing team
    if delta_team_A < 0: adjustment_factors_A = 1 - beta * (team_A_avg - team_A)
    if delta_team_B < 0: adjustment_factors_B = 1 - beta * (team_B_avg - team_B)
    
    # Calculate new ratings
    adjustment_delta_team_A = delta_team_A * adjustment_factors_A * np.array([role_weights['goalkeeper'], role_weights['attacker']])
    adjustment_delta_team_B = delta_team_B * adjustment_factors_B * np.array([role_weights['goalkeeper'], role_weights['attacker']])
    
    # Clip ratings to be within the 0 to 10 range
    new_ratings_A = np.clip(team_A + adjustment_delta_team_A, 0, 10)
    new_ratings_B = np.clip(team_B + adjustment_delta_team_B, 0, 10)
    
    return new_ratings_A.tolist(), new_ratings_B.tolist(),adjustment_delta_team_A.tolist(), adjustment_delta_team_B.tolist()


In [56]:
# Define initial ratings for both teams.
team_A = [7.0, 7.0]  # [goalkeeper, attacker]
team_B = [8.5, 6.0]  # [goalkeeper, attacker]

# Suppose Team A wins with a score of 10-2.
score_A = 9
score_B = 9

new_team_A, new_team_B = update_ratings(team_A, team_B, score_A, score_B)

delta_team_A = [new_team_A[i] - team_A[i] for i in range(len(team_A))]
delta_team_B = [new_team_B[i] - team_B[i] for i in range(len(team_B))]

print("Delta Team A Ratings:", delta_team_A)
print("Delta Team B Ratings:", delta_team_B)



Delta Team A Ratings: [0.049022749931009635, 0.049022749931009635]
Delta Team B Ratings: [-0.055150593672385284, -0.042894906189633986]


In [18]:
team_A = {'goalkeeper': 2.0, 'attacker': 7.0}
team_B = {'goalkeeper': 8.5, 'attacker': 6.0}

# Suppose Team A wins with a score of 10-6.
score_A = 9
score_B = 9

new_team_A, new_team_B = update_ratings(team_A, team_B, score_A, score_B)

delta_team_A = {role: new_team_A[role] - team_A[role] for role in team_A}
delta_team_B= {role: new_team_B[role] - team_B[role] for role in team_B}

print("Delta Team A Ratings:", delta_team_A)
print("Delta Team A Ratings:", delta_team_B)

Delta Team A Ratings: {'goalkeeper': 0.21797338379870812, 'attacker': 0.1307840302792247}
Delta Team A Ratings: {'goalkeeper': -0.15258136865909577, 'attacker': -0.19617604541883704}


In [8]:
team_A = {'goalkeeper': 2.0, 'attacker': 7.0}
team_B = {'goalkeeper': 8.5, 'attacker': 6.0}

# Suppose Team A wins with a score of 10-6.
score_A = 9
score_B = 3

new_team_A, new_team_B = update_ratings(team_A, team_B, score_A, score_B)

delta_team_A = {role: new_team_A[role] - team_A[role] for role in team_A}
delta_team_B= {role: new_team_B[role] - team_B[role] for role in team_B}

print("Delta Team A Ratings:", delta_team_A)
print("Delta Team A Ratings:", delta_team_B)

Delta Team A Ratings: {'goalkeeper': 1.2140349498052645, 'attacker': 0.7284209698831585}
Delta Team A Ratings: {'goalkeeper': -0.8498244648636852, 'attacker': -1.0926314548247378}


In [20]:
import math

def update_ratings(team_A, team_B, score_A, score_B, K=0.35, d=1.0, beta=0.1, role_weights={'goalkeeper': 1.0, 'attacker': 1.0}):
    """
    Update player ratings for a table football match with individual adjustments.
    
    Each team consists of 2 players with roles 'goalkeeper' and 'attacker'.
    Ratings are scaled from 0 to 10.
    
    The update now includes an adjustment based on the difference between each player's
    rating and the team’s average rating. For winners, players below the team average receive
    a larger boost, while players above the average gain less. For losers, players above the team
    average lose more points, while those below lose less.
    
    Parameters:
    - team_A: dict, e.g., {'goalkeeper': 2.0, 'attacker': 7.0}
    - team_B: dict, e.g., {'goalkeeper': 8.5, 'attacker': 6.0}
    - score_A: int, goals scored by Team A
    - score_B: int, goals scored by Team B
    - K: float, constant determining maximum change per match (default: 0.35)
    - d: float, scaling factor for the expected outcome formula (default: 1.0)
    - beta: float, adjustment factor for individual rating difference (default: 0.1)
    - role_weights: dict, optional custom weights for each role,
                    e.g., {'goalkeeper': 1.1, 'attacker': 0.9}. Default weights are 1.
                    
    Returns:
    - new_team_A: dict with updated ratings for Team A
    - new_team_B: dict with updated ratings for Team B
    """
    
    # Step 1: Calculate the team average ratings.
    team_A_avg = (team_A['goalkeeper'] + team_A['attacker']) / 2
    team_B_avg = (team_B['goalkeeper'] + team_B['attacker']) / 2

    # Step 2: Calculate the expected outcome using a logistic function.
    exponent = (team_B_avg - team_A_avg) / d
    E_A = 1 / (1 + math.pow(10, exponent))
    E_B = 1 - E_A

    # Step 3: Determine the actual match result.
    if score_A > score_B:
        S_A = 1
        S_B = 0
    elif score_B > score_A:
        S_A = 0
        S_B = 1
    else:
        S_A = 0.5
        S_B = 0.5

    goal_difference = abs(score_A - score_B)
    
    # Step 4: Compute the margin multiplier M using the logarithm of the goal difference plus one.
    # Use M = 1 if there is no goal difference (to avoid a multiplier of 0).
    M = math.log(goal_difference + 1) if goal_difference > 0 else 1

    # Step 5: Calculate the base team rating change using the formula:
    # ΔR_team = K * M * (S - E)
    delta_team_A = K * M * (S_A - E_A)
    delta_team_B = K * M * (S_B - E_B)
    
    # Step 6: Update each player's rating individually with adjustment factors.
    new_team_A = {}
    new_team_B = {}
    
    # For winners, use: adjustment_factor = 1 + beta*(TeamAvg - rating)
    # For losers, use: adjustment_factor = 1 - beta*(TeamAvg - rating)
    # For ties, use no extra adjustment (i.e., adjustment_factor = 1)
    for role, rating in team_A.items():
        if S_A > 0.5:  # winning team
            adjustment_factor = 1 + beta * (team_A_avg - rating)
        elif S_A < 0.5:  # losing team
            adjustment_factor = 1 - beta * (team_A_avg - rating)
        else:
            adjustment_factor = 1  # tie
        
        new_rating = rating + delta_team_A * adjustment_factor * role_weights.get(role, 1.0)
        new_team_A[role] = max(0, min(10, new_rating))
    
    for role, rating in team_B.items():
        if S_B > 0.5:  # winning team
            adjustment_factor = 1 + beta * (team_B_avg - rating)
        elif S_B < 0.5:  # losing team
            adjustment_factor = 1 - beta * (team_B_avg - rating)
        else:
            adjustment_factor = 1
        
        new_rating = rating + delta_team_B * adjustment_factor * role_weights.get(role, 1.0)
        new_team_B[role] = max(0, min(10, new_rating))
    
    return new_team_A, new_team_B

# Example usage:
if __name__ == "__main__":
    # Define initial ratings for both teams.
    team_A = {'goalkeeper': 4.0, 'attacker': 7.0}
    team_B = {'goalkeeper': 5.5, 'attacker': 6.0}
    
    # Suppose Team A wins with a score of 10-2.
    score_A = 9
    score_B = 9
    
    new_team_A, new_team_B = update_ratings(team_A, team_B, score_A, score_B)
    
    delta_team_A = {role: new_team_A[role] - team_A[role] for role in team_A}
    delta_team_B = {role: new_team_B[role] - team_B[role] for role in team_B}
    
    print("Delta Team A Ratings:", delta_team_A)
    print("Delta Team B Ratings:", delta_team_B)


Delta Team A Ratings: {'goalkeeper': 0.049022749931009635, 'attacker': 0.049022749931009635}
Delta Team B Ratings: {'goalkeeper': -0.049022749931009635, 'attacker': -0.049022749931009635}
